# CMDPにおける違反なしの探索

参考
* [Learning Policies with Zero or Bounded Constraint Violation for Constrained MDPs](https://arxiv.org/abs/2106.02684)

CMDPで制約違反をせずにリグレットを抑えるアルゴリズムを考えましょう．

表記
* 論文参照．また，[RL_CMDP_explore_exploit.ipynb](RL_CMDP_explore_exploit.ipynb)とだいたい同じ
* $\pi^0$は$V_1^{\pi^0}(\mu ; c, P)=c^0<\tau$を満たすときにStrictly safeといいます．

問題設定

面倒なので報酬とコストは既知とします．
次のCMDPを考えます．
$$
\max _\pi V_1^\pi(\mu ; r, P) \quad \text { s.t. } \quad V_1^\pi(\mu ; c, P) \leq \tau \text {. }
$$

## 違反なしのCMDP

エージェントはstrictly safeな$\pi^0$を知っているとします．
また，そのコスト$c^0:=V_1^{\pi^0}(\mu ; c, P)$を知っているとします．

アルゴリズムの性能を次のリグレットで測定します．
$$
\operatorname{Reg}(K ; r):=\sum_{k=1}^K\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^k}(\mu ; r, P)\right)
$$

また，次のように違反しないことを確率$1-\delta$以上で保証したいとします．

$$
\mathbb{P}\left(V_1^{\pi^k}(\mu ; c, P) \leq \tau, \forall k \in[K]\right) \geq 1-\delta
$$

---

**アルゴリズムの準備**

基本的には報酬にOptimismを追加し，コストにPessimismを追加するだけです．
まず，

$$
\hat{P}_h^k\left(s^{\prime} \mid s, a\right):=\frac{\sum_{k^{\prime}=1}^{k-1} \mathbb{1}\left(S_h^{k^{\prime}}=s, A_h^{k^{\prime}}=a, S_{h+1}^{k^{\prime}}=s^{\prime}\right)}{N_h^k(s, a) \vee 1},
$$

とします．
また，ボーナスを

$$
\beta_h^k(s, a):=\sqrt{\frac{1}{N_h^k(s, a) \vee 1} Z}, \quad \text { where } Z:=\log \left(16|\mathcal{S}|^2|\mathcal{A}| H K / \delta\right)
$$

とします．
報酬を

$$\bar{r}_h^k(s, a):={r}_h(s, a)+\alpha_r \beta_h^k(s, a), \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]$$

とます．ここで$\alpha_r:=1+|\mathcal{S}| H+\frac{4 H(1+|\mathcal{S}| H)}{\tau-c^0}$です．

コストは
$$
\underline{\mathrm{c}}_h^k(s, a):={c}_h(s, a)+(1+H|\mathcal{S}|) \beta_h^k(s, a), \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
$$
とします．

そして，方策を

$$
\Pi^k:= \begin{cases}\left\{\pi^0\right\} & \text { if } V_1^{\pi^0}\left(\mu ; \underline{\mathbf{c}}^k, \hat{P}^k\right) \geq\left(\tau+c^0\right) / 2 \\ \left\{\pi: V_1^\pi\left(\mu ; \underline{\mathbf{c}}^k, \hat{P}^k\right) \leq \tau\right\} & \text { otherwise. }\end{cases}
$$

として選択します．
つまり，$\pi^0$が$V_1^{\pi^0}\left(\mu ; \underline{\mathrm{c}}^k, \hat{P}^k\right)<\left(\tau+c^0\right) / 2$を満たすまで選択され続けます．
これは$\pi^0$が$\underline{\mathrm{c}}^k, \hat{P}^k$において安全である状況です．
$\tau \leq c^0$であり，$\pi^0$が訪れる範囲の精度がどんどん向上していくので，これはいつか満たされます．

これは$\left\{\pi: V_1^\pi\left(\mu ; \underline{\mathrm{c}}^k, \hat{P}^k\right) \leq \tau\right\}$は空でないことの十分条件です（最低でも$\pi^0$は含まれます）．

---

**アルゴリズム**

OptPress-LPアルゴリズムでは次を実行します：

1. for $k=1, 2, \dots, K$
2. $\hat{P}^k$を計算します
3. $\bar{r}^k, \underline{\mathrm{c}}^k$, and $\Pi^k$を計算します
4. $\pi^k \in \arg \max _{\pi \in \Pi^k} V_1^\pi\left(\mu ; \bar{r}^k, \hat{P}^k\right)$を計算します．
5. データを集めます

## リグレットバウンド

次が成立します：

OptPress-LPアルゴリズムは，確率$1 - \delta$以上で，次のリグレットを違反無しで達成する：

$$
\operatorname{Reg}^{\text {OPLP }}(K ; r)=\tilde{\mathcal{O}}\left(\frac{H^3}{\tau-c^0} \sqrt{|\mathcal{S}|^3|\mathcal{A}| K}+\frac{H^5|\mathcal{S}|^3|\mathcal{A}|}{\left(\tau-c^0\right)^2 \wedge\left(\tau-c^0\right)}\right) \text {, where } a \wedge b:=\min \{a, b\}
$$

**証明**

まず，リグレットを分解します：

$$
\begin{aligned}
\operatorname{Reg}^{\mathbf{O P L P}}(K ; r) & =\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|=1\right)\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^0}(\mu ; r, P)\right) \\
& +\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|>1\right)\left(V_1^{\pi^*}(\mu ; r, P)-V_1^{\pi^k}\left(\mu ; \bar{r}^k, \hat{P}^k\right)\right) \\
& +\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|>1\right)\left(V_1^{\pi^k}\left(\mu ; \bar{r}^k, \hat{P}^k\right)-V_1^{\pi^k}(\mu ; r, P)\right)
\end{aligned}
$$

最初の項はつまり$\pi^0$しか選択できない場合ですね．
１つ目の項は次でバウンドできます：

---

**補題**

確率$(1-\delta)$以上で，

$$
\sum_{k=1}^K \mathbb{1}\left(\left|\Pi^k\right|=1\right) \leq C^{\prime}
$$

が成立し，ここで，$C'=\tilde{\mathcal{O}}\left(H^4|\mathcal{S}|^3|\mathcal{A}| /\left(\left(\tau-c^0\right)^2 \wedge\left(\tau-c^0\right)\right)\right)$です．

